<a href="https://colab.research.google.com/github/Paul-mwaura/Programming-PyTorch-for-Deep-Learning/blob/main/Introduction_Image_Classification_using_Pytorch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Introduction Image Classification using Pytorch

The image processing using Pytorch implement on the MNIST data set. This dataset contains handwritten digits of the 10 classes from 0 to 9. The task is we have classified the images of digits. In the MNIST dataset, we have images of digits that were taken from a variety of scanned documents, normalized in size, and centered. Subsequently, each image is a 28 by 28-pixel square (784 pixels total). A standard split of the dataset is used to evaluate and compare models. There are 60,000 images are used to train a model and a separate set of 10,000 images are used to test it.

### Import required library

In [2]:
# Import required library
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim

# version of pytorch
print(torch.__version__)


# transformations to be applied on images
transform = transforms.Compose([transforms.ToTensor(),
                          	transforms.Normalize((0.5,), (0.5,)),
                          	])

# defining the training and testing set
trainset = datasets.MNIST('./data', download=True, train=True, transform=transform)
testset = datasets.MNIST('./', download=True, train=False, transform=transform)

1.8.1+cu101
Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw

Processing...


...

Done!
Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./MNIST/raw/train-images-idx3-ubyte.gz to ./MNIST/raw




Extracting ./MNIST/raw/train-labels-idx1-ubyte.gz to ./MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting ./MNIST/raw/t10k-images-idx3-ubyte.gz to ./MNIST/raw




Extracting ./MNIST/raw/t10k-labels-idx1-ubyte.gz to ./MNIST/raw

Processing...
Done!


Next, I have defined the train and test loader which will help us to load the training and test set in batches. I will define the batch size as 64

### defining trainloader and testloader

In [5]:
# defining trainloader and testloader
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

# shape of training data
dataiter = iter(trainloader)
images, labels = dataiter.next()

print(images.shape)
print(labels.shape)

# defining the model architecture
class Net(nn.Module):  
 def __init__(self):
     super(Net, self).__init__()
 
     self.cnn_layers = nn.Sequential(
         # Defining a 2D convolution layer
         nn.Conv2d(1, 4, kernel_size=3, stride=1, padding=1),
         nn.BatchNorm2d(4),
         nn.ReLU(inplace=True),
         nn.MaxPool2d(kernel_size=2, stride=2),
         # Defining another 2D convolution layer
         nn.Conv2d(4, 4, kernel_size=3, stride=1, padding=1),
         nn.BatchNorm2d(4),
         nn.ReLU(inplace=True),
         nn.MaxPool2d(kernel_size=2, stride=2),
     )
 
     self.linear_layers = nn.Sequential(
         nn.Linear(4 * 7 * 7, 10)
     )
 
 # Defining the forward pass   
 def forward(self, x):
     x = self.cnn_layers(x)
     x = x.view(x.size(0), -1)
     x = self.linear_layers(x)
     return x

torch.Size([64, 1, 28, 28])
torch.Size([64])


### defining the model

In [6]:
# defining the model
model = Net()
# defining the optimizer
optimizer = optim.Adam(model.parameters(), lr=0.01)
# defining the loss function
criterion = nn.CrossEntropyLoss()
# checking if GPU is available
if torch.cuda.is_available():
   model = model.cuda()
   criterion = criterion.cuda()
  
print(model)

Net(
  (cnn_layers): Sequential(
    (0): Conv2d(1, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(4, 4, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(4, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU(inplace=True)
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (linear_layers): Sequential(
    (0): Linear(in_features=196, out_features=10, bias=True)
  )
)


### Train the CNN Model

In [7]:
for i in range(10):
   running_loss = 0
   for images, labels in trainloader:
 
       if torch.cuda.is_available():
         images = images.cuda()
         labels = labels.cuda()
 
       # Training pass
       optimizer.zero_grad()
      
       output = model(images)
       loss = criterion(output, labels)
      
       #This is where the model learns by backpropagating
       loss.backward()
      
       #And optimizes its weights here
       optimizer.step()
      
       running_loss += loss.item()
   else:
       print("Epoch {} - Training loss: {}".format(i+1, running_loss/len(trainloader)))

Epoch 1 - Training loss: 0.16872656090494825
Epoch 2 - Training loss: 0.09250285256820828
Epoch 3 - Training loss: 0.08368386059422601
Epoch 4 - Training loss: 0.07861704244548931
Epoch 5 - Training loss: 0.07534394899024757
Epoch 6 - Training loss: 0.072253399530797
Epoch 7 - Training loss: 0.07101372641120066
Epoch 8 - Training loss: 0.06668884910495161
Epoch 9 - Training loss: 0.0655440903548747
Epoch 10 - Training loss: 0.06437008913833775


You can see that the training is decreasing with an increasing number of epochs. This means that our model is learning patterns from the training set. Let’s check the performance of this model on the test set:

### getting predictions on test set and measuring the performance

In [8]:
# getting predictions on test set and measuring the performance
correct_count, all_count = 0, 0
for images,labels in testloader:
 for i in range(len(labels)):
   if torch.cuda.is_available():
       images = images.cuda()
       labels = labels.cuda()
   img = images[i].view(1, 1, 28, 28)
   with torch.no_grad():
       logps = model(img)
 
  
   ps = torch.exp(logps)
   probab = list(ps.cpu()[0])
   pred_label = probab.index(max(probab))
   true_label = labels.cpu()[i]
   if(true_label == pred_label):
     correct_count += 1
   all_count += 1
 
print("Number Of Images Tested =", all_count)
print("\nModel Accuracy =", (correct_count/all_count))

Number Of Images Tested = 10000

Model Accuracy = 0.9438


So, we tested a total of 10000 images and the model is around 96% accurate in predicting the labels for test images. This is how you can build a Convolutional Neural Network in PyTorch. In the next section, we will look at how to implement the same architecture in TensorFlow.